## Importing

In [ ]:
%%capture
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import gc #garbage collector for gpu memory 
from tqdm import tqdm
import json
import datetime as dt
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig, AutoModel
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Loading Data

In [ ]:
politifact_data = json.load(open("gdrive/MyDrive/BT4222/Data/politifact_clean.json", "r"))
gossipcop_data = json.load(open("gdrive/MyDrive/BT4222/Data/gossipcop_clean.json", "r"))

In [ ]:
politifact_df = pd.DataFrame(politifact_data)
gossipcop_df = pd.DataFrame(gossipcop_data)

In [ ]:
politifact_df['target'] = politifact_df['label'].apply(lambda x: 1 if x=='real' else 0)
gossipcop_df['target'] = gossipcop_df['label'].apply(lambda x: 1 if x=='real' else 0)

In [ ]:
politifact_df['parsed_month'] = politifact_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%m") if not pd.isna(x) else '0')
gossipcop_df['parsed_month'] = gossipcop_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%m") if not pd.isna(x) else '0')

In [ ]:
politifact_df['parsed_hour'] = politifact_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%H") if not pd.isna(x) else '0')
gossipcop_df['parsed_hour'] = gossipcop_df['publish_date'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime("%H") if not pd.isna(x) else '0')

In [ ]:
politifact_df['publisher'] = politifact_df['publisher'].fillna('None')
gossipcop_df['publisher'] = gossipcop_df['publisher'].fillna('None')

## Choose type of article to run

In [ ]:
# Choose gossipcop or politifact
article = "gossipcop"

In [ ]:
if article == "politifact":
  X_extra = politifact_df[['parsed_hour','parsed_month', 'publisher']]
else:
  X_extra = gossipcop_df[['parsed_hour','parsed_month', 'publisher']]

## Summarizing

In [ ]:
%%capture
!pip install sumy
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from gensim.summarization import summarize
from sumy.utils import get_stop_words
from sumy.nlp.tokenizers import Tokenizer as sumytoken
from sumy.nlp.stemmers import Stemmer
from sumy.summarizers.luhn import LuhnSummarizer

# For TextRank
import nltk
nltk.download('punkt') # one time execution

In [ ]:
def luhn_summarizer(text, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, sumytoken(LANGUAGE))
    sentences = []
    for sentence in summarizer_luhn(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)

In [ ]:
LANGUAGE = "english"
SENTENCES_COUNT = 15 # dk how to collaborate this
stemmer = Stemmer(LANGUAGE)
summarizer_luhn = LuhnSummarizer(stemmer)
summarizer_luhn.stop_words = get_stop_words(LANGUAGE)

In [ ]:
gossipcop_df['text_summarised'] = gossipcop_df['text'].apply(lambda x: luhn_summarizer(x, LANGUAGE,SENTENCES_COUNT))
politifact_df['text_summarised'] = politifact_df['text'].apply(lambda x: luhn_summarizer(x, LANGUAGE,SENTENCES_COUNT))

In [ ]:
politifact_df['text_summarised_len'] = politifact_df['text_summarised'].str.split().str.len()
gossipcop_df['text_summarised_len'] = gossipcop_df['text_summarised'].str.split().str.len()

# Tokenizing

In [ ]:
def tokenize(df):
    # Get tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    # tokenize text
    print("Title Tokenizing")
    title_tokenized_df = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:98] + ['[SEP]'], tqdm(df['title'])))
    # Get token index
    title_indexed_tokens = list(map(tokenizer.convert_tokens_to_ids, title_tokenized_df))
    
    # Pad tokens
    totalpadlength = 100
    title_index_padded = np.array([xi+[0]*(totalpadlength-len(xi)) for xi in title_indexed_tokens])

    # Mask
    title_mask_variable = [[float(i>0) for i in ii] for ii in title_index_padded]


    print("Article Tokenizing")
    article_tokenized_df = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], tqdm(df['text_summarised'])))    
    article_indexed_tokens = list(map(tokenizer.convert_tokens_to_ids, article_tokenized_df))
  
    # Pad tokens
    totalpadlength = 512
    article_index_padded = np.array([xi+[0]*(totalpadlength-len(xi)) for xi in article_indexed_tokens])

    # Mask
    article_mask_variable = [[float(i>0) for i in ii] for ii in article_index_padded]

    # Target Variable
    target_variable = df['target'].values

    return title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, target_variable

def format_tensors(article_data, article_mask, title_data, title_mask, extra_features, labels, batch_size):
    
    X_article = torch.from_numpy(article_data)
    X_article = X_article.long()
    article_mask = torch.tensor(article_mask)

    X_title = torch.from_numpy(title_data)
    X_title = X_title.long()
    title_mask = torch.tensor(title_mask)

    extra_features = torch.from_numpy(extra_features)
    extra_features = extra_features.long()

    y = torch.from_numpy(labels)
    y = y.long()

    tensordata = data_utils.TensorDataset(X_article, article_mask, X_title, title_mask, extra_features,  y)
    loader = data_utils.DataLoader(tensordata, batch_size=batch_size, shuffle=False)
    
    return loader

def train_validation_test(title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, X_extra, target_variable, BATCH_SIZE = 8):
    # Train test split for train set
    X_train_title, X_rest_title, y_train, y_rest = train_test_split(title_index_padded, target_variable, test_size=0.3, random_state=42)
    train_masks_title, rest_masks_title, _, _ = train_test_split(title_mask_variable, title_index_padded, test_size=0.3, random_state=42)

    # Train test split again for validation and test set
    X_val_title, X_test_title, y_val, y_test = train_test_split(X_rest_title, y_rest, test_size=0.5, random_state=42)
    val_masks_title, test_masks_title, _, _ = train_test_split(rest_masks_title, X_rest_title, test_size=0.5, random_state=42)

    # Train test split for train set
    X_train_article, X_rest_article, _, _ = train_test_split(article_index_padded, target_variable, test_size=0.3, random_state=42)
    train_masks_article, rest_masks_article, _, _ = train_test_split(article_mask_variable, article_index_padded, test_size=0.3, random_state=42)

    # Train test split again for validation and test set
    X_val_article, X_test_article, _, _ = train_test_split(X_rest_article, y_rest, test_size=0.5, random_state=42)
    val_masks_article, test_masks_article, _, _ = train_test_split(rest_masks_article, X_rest_article, test_size=0.5, random_state=42)

    # Train test split for train set
    X_train_extra, X_rest_extra, _, _ = train_test_split(X_extra, target_variable, test_size=0.3, random_state=42)

    # Train test split again for validation and test set
    X_val_extra, X_test_extra, _, _ = train_test_split(X_rest_extra, y_rest, test_size=0.5, random_state=42)

    trainloader = format_tensors(X_train_article, train_masks_article, X_train_title, train_masks_title, X_train_extra, y_train, BATCH_SIZE)
    validationloader = format_tensors(X_val_article, val_masks_article, X_val_title, val_masks_title, X_val_extra, y_val, BATCH_SIZE)
    testloader = format_tensors(X_test_article, test_masks_article, X_test_title, test_masks_title, X_test_extra, y_test, BATCH_SIZE)

    return trainloader, validationloader, testloader

## Processing Other Features

We shall one hot encode hour, month and publisher. Fill publisher NaN with None text.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_extra['parsed_hour'] = X_extra['parsed_hour'].astype('str')
X_extra['parsed_month'] = X_extra['parsed_month'].astype('str')
X_extra['publisher'] = X_extra['publisher'].astype('str')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(X_extra)
X_extra_enc = enc.transform(X_extra).toarray()

In [ ]:
title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, target_variable = tokenize(politifact_df) if article == "politifact" else tokenize(gossipcop_df)
trainloader, validationloader, testloader = train_validation_test(title_index_padded, title_mask_variable, article_index_padded, article_mask_variable, X_extra_enc, target_variable)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Title Tokenizing


100%|██████████| 20049/20049 [00:09<00:00, 2152.16it/s]


Article Tokenizing


100%|██████████| 20049/20049 [02:44<00:00, 121.85it/s]


# Model Definition

In [ ]:
class FrozenLearningAndFeatures(torch.nn.Module):
    """
    This takes a transformer backbone and puts a slightly-modified classification head on top.
    
    """

    def __init__(self, article_model_name, title_model_name, extra_dims, num_labels):
        # num_extra_dims corresponds to the number of extra dimensions of numerical/categorical data

        super().__init__()

        # self.config = AutoConfig.from_pretrained(model_name)
        self.article_transformer = AutoModel.from_pretrained(article_model_name) #article transformer
        self.title_transformer = AutoModel.from_pretrained(title_model_name) #title transformer

        ## Freezing Bert Params so as not to do backprop directly ##
        # for param in self.article_transformer.parameters():
        #   param.requires_grad = False
        
        for param in self.title_transformer.parameters():
          param.requires_grad = False
        
        num_hidden_size_1 = self.article_transformer.config.hidden_size
        num_hidden_size_2 = self.title_transformer.config.hidden_size # May be different depending on which model you use. Common sizes are 768 and 1024. Look in the config.json file 
        
        input_dim = num_hidden_size_1+num_hidden_size_2 + extra_dims
        
        self.linear = torch.nn.Linear(input_dim, 100)
        self.classifier = torch.nn.Linear(100, 2)
        self.dropout = torch.nn.Dropout(0.25)

    def forward(self, article_input_ids, article_attention_mask, title_input_ids, title_attention_mask, extra_features, labels=None):
        """
        extra_data should be of shape [batch_size, dim] 
        where dim is the number of additional numerical/categorical dimensions
        """

        article_hidden_states = self.article_transformer(input_ids=article_input_ids, attention_mask=article_attention_mask) # [batch size, sequence length, hidden size]

        article_cls_embeds = article_hidden_states.last_hidden_state[:, 0, :] # [batch size, hidden size]

        title_hidden_states = self.title_transformer(input_ids=title_input_ids, attention_mask=title_attention_mask) # [batch size, sequence length, hidden size]

        title_cls_embeds = title_hidden_states.last_hidden_state[:, 0, :] # [batch size, hidden size]

        concat = torch.cat((article_cls_embeds, title_cls_embeds,extra_features), dim=-1) # [batch size, hidden size+num extra dims]
        x = self.linear(concat)
        x = self.dropout(x) 
        output = self.classifier(x) # [batch size, num labels]

        return output

# Model Creation

In [ ]:
def train_model(epochs, model, learning_rate, start_from_epoch=1):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache() #memory
    gc.collect() #memory
    NUM_EPOCHS = epochs
    loss_function = nn.CrossEntropyLoss()
    losses = []
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(start_from_epoch, NUM_EPOCHS+1):
        model.train()

        # For epoch metrics
        epoch_loss = 0.0
        preds, truth, pred_proba = [],[],[]
        iteration = 0
        for i, batch in enumerate(tqdm(trainloader)):
            iteration += 1
            article_token_ids, article_masks, title_token_ids, title_masks, extra_features, labels = tuple(t.to(device) for t in batch)
            optimizer.zero_grad()
            outputs = model(article_token_ids, article_masks, title_token_ids, title_masks, extra_features, labels)
            loss = loss_function(outputs, labels)
            epoch_loss += float(loss.item())
            yhat = outputs
            loss.backward()
            optimizer.step()

            # Metrics for batch
            prediction_proba = torch.sigmoid(yhat[:,1]).cpu().data.numpy()
            prediction = (prediction_proba > 0.5).astype(int)
            baseline = labels.long().cpu().data.numpy().astype(int)
            preds.extend(prediction)
            pred_proba.extend(prediction_proba)
            truth.extend(baseline)

            del article_token_ids, article_masks, title_token_ids, title_masks, labels #memory
            torch.cuda.empty_cache() #memory
            gc.collect() #memory
        

        # Calculate train and validation metrics and log them
        with torch.set_grad_enabled(False):
            metrics = {}
            # Training
            avg_accuracy, avg_roc_auc, avg_f1, avg_loss = accuracy_score(truth, preds), roc_auc_score(truth, pred_proba), f1_score(truth, preds), epoch_loss/float(iteration)
            print(f'Epoch {epoch}:\nTraining Accuracy: 'f'{avg_accuracy:.2f}%')
            print(f'Training ROC AUC: 'f'{avg_roc_auc:.2f}%')
            print(f'Training F1: 'f'{avg_f1:.2f}%')
            print(f'Training loss: 'f'{avg_loss}%\n')
            metrics['train'] = {
                'accuracy':avg_accuracy,
                'roc_auc':avg_roc_auc,
                'f1':avg_f1,
                'loss':avg_loss
            }

            # Validation
            model.eval()
            epoch_loss = 0.0
            preds, truth, pred_proba = [],[],[]
            iteration = 0
            with torch.no_grad():
                for i, batch in enumerate(tqdm(validationloader)):
                    iteration += 1
                    article_token_ids, article_masks, title_token_ids, title_masks, extra_features, labels = tuple(t.to(device) for t in batch)
                    outputs = model(article_token_ids, article_masks, title_token_ids, title_masks, extra_features, labels)
                    
                    loss = loss_function(outputs, labels)
                    epoch_loss += float(loss.item())
                    yhat = outputs

                    prediction_proba = torch.sigmoid(yhat[:,1]).cpu().data.numpy()
                    prediction = (prediction_proba > 0.5).astype(int)
                    baseline = labels.long().cpu().data.numpy().astype(int)
                    preds.extend(prediction)
                    pred_proba.extend(prediction_proba)
                    truth.extend(baseline)

                    del article_token_ids, article_masks, title_token_ids, title_masks, extra_features, labels #memory
                    torch.cuda.empty_cache() #memory
                    gc.collect() #memory

            avg_accuracy, avg_roc_auc, avg_f1, avg_loss = accuracy_score(truth, preds), roc_auc_score(truth, pred_proba), f1_score(truth, preds), epoch_loss/float(iteration)
            print(f'Validation Accuracy: 'f'{avg_accuracy:.2f}%')
            print(f'Validation ROC AUC: 'f'{avg_roc_auc:.2f}%')
            print(f'Validation F1: 'f'{avg_f1:.2f}%')
            print(f'Validation loss: 'f'{avg_loss}%\n')
            metrics['validation'] = {
                'accuracy':avg_accuracy,
                'roc_auc':avg_roc_auc,
                'f1':avg_f1,
                'loss':avg_loss
            }


In [ ]:
article_model_name = f"distilbert-base-uncased"
title_model_name = f"distilbert-base-uncased"
# Path to the best title distilbert model we trained
# title_model_name = f"gdrive/MyDrive/BT4222/Code/machine_learning/xp/{article}/distilbert_title/model_epoch2" 
num_labels = 2
num_extra_dims = 1512

model = FrozenLearningAndFeatures(article_model_name, title_model_name, num_extra_dims, num_labels=2)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at gdrive/MyDrive/BT4222/Code/machine_learning/xp/gossipcop/distilbert_title/model_epoch2 were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
- Th

In [ ]:
train_model(10, model, 3e-5, start_from_epoch=1)

100%|██████████| 1755/1755 [41:30<00:00,  1.42s/it]


Epoch 1:
Training Accuracy: 0.92%
Training ROC AUC: 0.96%
Training F1: 0.95%
Training loss: 0.21536550455092535%



100%|██████████| 376/376 [04:05<00:00,  1.53it/s]


Validation Accuracy: 0.85%
Validation ROC AUC: 0.89%
Validation F1: 0.90%
Validation loss: 0.39232140566727663%



100%|██████████| 1755/1755 [41:10<00:00,  1.41s/it]


...

100%|██████████| 1755/1755 [41:38<00:00,  1.42s/it]


Epoch 8:
Training Accuracy: 0.96%
Training ROC AUC: 0.99%
Training F1: 0.98%
Training loss: 0.10075911798377116%



100%|██████████| 376/376 [04:09<00:00,  1.51it/s]


Validation Accuracy: 0.86%
Validation ROC AUC: 0.89%
Validation F1: 0.91%
Validation loss: 0.45608115046714837%



 28%|██▊       | 484/1755 [11:33<30:38,  1.45s/it]